In [1]:
# set workspace
WhoIsWorking = "AA"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = r'C:\Users\achan\OneDrive\Desktop\Term Project Data'
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

C:\Users\achan\OneDrive\Desktop\Term Project Data


In [2]:
import os

# List all files in the folder
files = os.listdir(folder_path)

# Print the list of files
for file in files:
    print(file)

Archive
AverageHomePrices_2023.csv
BuildingOutlines.CPG
BuildingOutlines.dbf
BuildingOutlines.prj
BuildingOutlines.sbn
BuildingOutlines.sbx
BuildingOutlines.shp
BuildingOutlines.shp.xml
BuildingOutlines.shx
BuildingOutlines_Dissolved.dbf
BuildingOutlines_Dissolved.prj
BuildingOutlines_Dissolved.sbn
BuildingOutlines_Dissolved.sbx
BuildingOutlines_Dissolved.shp
BuildingOutlines_Dissolved.shp.xml
BuildingOutlines_Dissolved.shx
CensusProfile_2021_PopAndIncome_Reformatted.csv
ClimateData_Hourly.csv
EnhancedPointsOfInterestPoint_Toronto.cpg
EnhancedPointsOfInterestPoint_Toronto.dbf
EnhancedPointsOfInterestPoint_Toronto.prj
EnhancedPointsOfInterestPoint_Toronto.sbn
EnhancedPointsOfInterestPoint_Toronto.sbx
EnhancedPointsOfInterestPoint_Toronto.shp
EnhancedPointsOfInterestPoint_Toronto.shp.xml
EnhancedPointsOfInterestPoint_Toronto.shx
HousePrices_Condos.csv
HousePrices_Detached.csv
HousePrices_SemiDetached.csv
info
LandUseClassified_20211220.cpg
LandUseClassified_20211220.dbf
LandUseClassified

In [3]:
import os
import geopandas as gpd
import pandas as pd

# Path to the shapefile for neighborhoods
neighborhoods_shapefile_path = os.path.join(folder_path, 'Neighbourhoods.shp')

# Read the shapefile into a GeoDataFrame
neighbourhoods_gdf = gpd.read_file(neighborhoods_shapefile_path)

# Print the first few lines of the  table
neighbourhoods_gdf.head(2)

,_id1,AREA_ID2,AREA_AT3,PARENT_4,AREA_SH5,AREA_LO6,AREA_NA7,AREA_DE8,CLASSIF9,CLASSIF10,OBJECTI11,geometry
0,1,2502366,26022881,0,174,174,South Eglinton-Davisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697..."
1,2,2502365,26022880,0,173,173,North Toronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"POLYGON ((-79.39744 43.70693, -79.39837 43.706..."


In [4]:
# Keep only certain fields and rename fields
neighbourhoods_gdf = neighbourhoods_gdf[['geometry', '_id1', 'AREA_NA7']].rename(columns={'_id1': 'UID', 'AREA_NA7': 'NeighbourhoodName'})

# Print the first few lines of the updated GeoDataFrame
neighbourhoods_gdf.head(2)

,geometry,UID,NeighbourhoodName
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto


In [ ]:
# Path to the shapefile for poles
poles_shapefile_path = os.path.join(folder_path, 'Poles.shp')

# Read shapefiles into GeoDataFrames
poles_gdf = gpd.read_file(poles_shapefile_path)

# List fields in the poles GeoDataFrame
poles_fields = poles_gdf.columns
print("Fields in the poles GeoDataFrame:")
print(poles_fields)

In [ ]:
# List unique attributes in the 'subtype' field of poles
unique_subtypes = poles_gdf['subtype'].unique()

# Print unique attributes
print("Unique attributes in the 'subtype' field of poles:")
print(unique_subtypes)

In [ ]:
# Filter poles to only keep certain subtypes
desired_subtypes = ['Street Light and Traffic Signal Pole', 'Street Light Pole', 'Pedestrian Light Pole']
filtered_poles_gdf = poles_gdf[poles_gdf['subtype'].isin(desired_subtypes)]

In [ ]:
# Perform spatial join to count the number of poles in each neighborhood
poles_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, filtered_poles_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of poles as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['Poles_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(poles_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

In [ ]:
# Path to the shapefile for POI
poi_shapefile_path = os.path.join(folder_path, 'EnhancedPointsOfInterestPoint_Toronto.shp')

# Read the shapefile into a GeoDataFrame
poi_gdf = gpd.read_file(poi_shapefile_path)

# Perform spatial join to count the number of POIs within each neighbourhood
poi_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, poi_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of POIs as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['POI_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(poi_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head()

In [ ]:
# Path to the shapefile for traffic cameras
traffic_cameras_shapefile_path = os.path.join(folder_path, 'TrafficCamera.shp')

# Read the shapefile into a GeoDataFrame
traffic_cameras_gdf = gpd.read_file(traffic_cameras_shapefile_path)

# Perform spatial join to count the number of traffic cameras within each neighborhood
traffic_cameras_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, traffic_cameras_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of traffic cameras as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['TCamera_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(traffic_cameras_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head()

In [ ]:
import pandas as pd

# Path to the CSV file
cenus_file_path = os.path.join(folder_path, 'CensusProfile_2021_PopAndIncome_Reformatted.csv')

# Read the CSV file into a DataFrame
census_df = pd.read_csv(cenus_file_path)

# Display the first few rows of the GeoDataFrame
census_df.head()

In [ ]:
neighbourhoods_gdf.head()

In [ ]:
# Merge the DataFrame with the neighborhoods GeoDataFrame based on the "NeighbourhoodName" column
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, census_df, left_on='NeighbourhoodName', right_on='Neighbourhood', how='left')

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

In [ ]:
# Drop columns
neighbourhoods_gdf.drop(['Geography', 'Neighbourhood'], axis=1, inplace=True)

# Display updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

In [ ]:
# Load the CSV file
Buildings_file = os.path.join(folder_path, "TI_Neighbourhoods_BuildingDissolved.csv")
Buildings_data = pd.read_csv(Buildings_file)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, Buildings_data, left_on="NeighbourhoodName", right_on="AREA_NA7")

In [ ]:
# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Specify columns to drop
columns_to_drop = ['OBJECTID', 'AREA_NA7', 'AREA']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Load the CSV file
ParkOSNA_file = os.path.join(folder_path, "TI_Neighbourhoods_ParksOpenSpaceNaturalArea.csv")
ParkOSNA_data = pd.read_csv(ParkOSNA_file)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, ParkOSNA_data, left_on="NeighbourhoodName", right_on="AREA_NA7")

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Specify columns to drop
columns_to_drop = ['OBJECTID', 'AREA_NA7', 'AREA']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Subtract "ParksOSNAPERCENTAGE" from 100 to get the "BuiltUpPERCENTAGE"
neighbourhoods_gdf['BuiltUpPERCENTAGE'] = 100 - neighbourhoods_gdf['ParksOSNAPERCENTAGE']

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Load avg home prices CSV file
AvgHomePrice = os.path.join(folder_path, "AverageHomePrices_2023.csv")
AvgHomePrice_data = pd.read_csv(AvgHomePrice)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, AvgHomePrice_data, left_on="NeighbourhoodName", right_on="Neighbourhood")

In [ ]:
# Specify columns to drop
columns_to_drop = ['UID', 'Neighbourhood', 'TREB_Code']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

In [ ]:
# Check for null values in each column
null_counts = neighbourhoods_gdf.isnull().sum()

# Display null counts for each column
print("Null counts for each column:")
print(null_counts)

In [ ]:
# Export the DataFrame to a CSV file
neighbourhoods_gdf.to_csv(os.path.join(folder_path, 'Neighbourhood_Cleaned.csv'), index=False)

In [ ]:
# Load the ClimateData_Hourly.csv file
ClimateData_file = os.path.join(folder_path, "ClimateData_Hourly.csv")
ClimateData_data = pd.read_csv(ClimateData_file)

# List all fields in the file
fields_in_file = ClimateData_data.columns.tolist()

# Display the list of fields
print(fields_in_file)

In [ ]:
# Keep only the 'UTC_DATE_FORMATTED' and 'TEMP' fields
ClimateData_data = ClimateData_data[['DATE_TIME_LINK', 'TEMP']]

# Display the updated DataFrame
ClimateData_data.head()

In [ ]:
# Convert 'TEMP' column to numeric values, coercing errors to NaN
ClimateData_data['TEMP'] = pd.to_numeric(ClimateData_data['TEMP'], errors='coerce')

# Define a function to categorize temperature
def categorize_temperature(temp):
    if pd.isnull(temp) or temp == "":
        return None  # If temperature is blank or null, return None
    elif temp <= -35:
        return "Extreme Cold"
    elif -35 < temp <= -20:
        return "Very Cold"
    elif -20 < temp <= -10:
        return "Cold"
    elif -10 < temp <= -5:
        return "Chilly"
    elif -5 < temp <= 0:
        return "Cool"
    elif 0 < temp <= 10:
        return "Mild"
    elif 10 < temp <= 20:
        return "Warm"
    elif 20 < temp <= 29:
        return "Hot"
    else:
        return "Very Hot"

# Create the TEMPCategory field using the categorize_temperature function
ClimateData_data['TEMPCategory'] = ClimateData_data['TEMP'].apply(categorize_temperature)

In [ ]:
# Display the first few rows to verify
print(ClimateData_data.head(10))

In [ ]:
# Load the CSV file
MajorCrime_file = os.path.join(folder_path, "MajorCrimeIndicators.csv")
MajorCrime_data = pd.read_csv(MajorCrime_file)

# List all fields in the file
fields_in_file = MajorCrime_data.columns.tolist()

# Display the list of fields
print(fields_in_file)

In [ ]:
# Find the number of records in the filtered data
num_records = len(MajorCrime_data)

# Display the number of records
print("Number of records:", num_records)

In [ ]:
# List unique attributes in the OCC_YEAR column
unique_years = MajorCrime_data['OCC_YEAR'].unique()

# Display the unique attributes
print("Unique years:", unique_years)

In [ ]:
# Filter the data where OCC_YEAR is between 2014 and 2023 (inclusive)
MajorCrime_data_filtered = MajorCrime_data[(MajorCrime_data['OCC_YEAR'] >= 2014) & (MajorCrime_data['OCC_YEAR'] <= 2023)]

# Display the filtered data
MajorCrime_data_filtered.head()

In [ ]:
# Find the number of records in the filtered data
num_records = len(MajorCrime_data_filtered)

# Display the number of records
print("Number of records:", num_records)

In [ ]:
# List unique attributes in the OCC_YEAR column
unique_years = MajorCrime_data_filtered['OCC_YEAR'].unique()

# Display the unique attributes
print("Unique years:", unique_years)

In [ ]:
# Filter out rows where either LONG_WGS84 or LAT_WGS84 equals 0
MajorCrime_data_filtered = MajorCrime_data_filtered[(MajorCrime_data_filtered['LONG_WGS84'] != 0) & (MajorCrime_data_filtered['LAT_WGS84'] != 0)]

# Find the number of records in the filtered data
num_records = len(MajorCrime_data_filtered)

# Display the number of records
print("Number of records:", num_records)

In [ ]:
# Define a function to convert month names to month numbers
def month_to_num(month):
    month_dict = {
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }
    return month_dict.get(month)

# Apply the function to create the new field
MajorCrime_data_filtered['MonthNum'] = MajorCrime_data_filtered['OCC_MONTH'].apply(month_to_num)

In [ ]:
# Display the DataFrame with the new field
MajorCrime_data_filtered

In [ ]:
# Convert OCC_YEAR, OCC_MONTH, and OCC_DAY to integers
MajorCrime_data_filtered['OCC_YEAR'] = MajorCrime_data_filtered['OCC_YEAR'].astype(int)
MajorCrime_data_filtered['OCC_DAY'] = MajorCrime_data_filtered['OCC_DAY'].astype(int)
MajorCrime_data_filtered['OCC_DOY'] = MajorCrime_data_filtered['OCC_DOY'].astype(int)

# Define a function to convert month names to month numbers
def month_to_num(month):
    month_dict = {
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }
    return month_dict.get(month)

# Define a function to create the date-time link
def create_date_time_link(year, month, day, hour):
    return f"{year}/{month}/{day} {hour}"

# Apply the function to create the new field
MajorCrime_data_filtered['MonthNum'] = MajorCrime_data_filtered['OCC_MONTH'].apply(month_to_num)
MajorCrime_data_filtered['DATE_TIME_LINK'] = MajorCrime_data_filtered.apply(lambda row: create_date_time_link(row['OCC_YEAR'], row['MonthNum'], row['OCC_DAY'], row['OCC_HOUR']), axis=1)

In [ ]:
# Display the DataFrame with the new field
MajorCrime_data_filtered

In [ ]:
# List of fields to keep
fields_to_keep = ['OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'LOCATION_TYPE', 'PREMISES_TYPE', 'OFFENCE', 'MCI_CATEGORY','NEIGHBOURHOOD_158', 'LONG_WGS84', 'LAT_WGS84', 'DATE_TIME_LINK']

# Keep only the desired fields using direct column indexing
MajorCrime_data_filtered = MajorCrime_data_filtered[fields_to_keep]

In [ ]:
# Display the updated DataFrame
MajorCrime_data_filtered

In [ ]:
# Define a function to categorize hours
def categorize_time(hour):
    if 6 <= hour <= 11:
        return 'Morning'
    elif 12 <= hour <= 17:
        return 'Afternoon'
    elif 18 <= hour <= 23:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create the new field
MajorCrime_data_filtered.loc[:,'TIMECategory'] = MajorCrime_data_filtered['OCC_HOUR'].apply(categorize_time)

In [ ]:
# Display the DataFrame with the new field
MajorCrime_data_filtered.head(10)

In [ ]:
# Merge the climate data with the crime data based on the date columns
MajorCrime_data_filtered = pd.merge(MajorCrime_data_filtered, ClimateData_data, left_on='DATE_TIME_LINK', right_on='DATE_TIME_LINK', how='left')

In [ ]:
# Display the merged data
MajorCrime_data_filtered.head()

In [ ]:
# Check for null values in each column
null_counts = MajorCrime_data_filtered.isnull().sum()

# Display null counts for each column
print("Null counts for each column:")
print(null_counts)

In [ ]:
# Display information about the DataFrame
print("Information about MajorCrime_data_filtered:")
MajorCrime_data_filtered.info()

In [ ]:
# Export the DataFrame to a CSV file
MajorCrime_data_filtered.to_csv(os.path.join(folder_path, 'MCI_Cleaned.csv'), index=False)